In [1]:
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os, logging

from eventor_api import EventorAPI

logging.basicConfig(level=logging.INFO)

load_dotenv()
api = EventorAPI(os.getenv('API_KEY'))

In [3]:
# LOAD INPUT DATA

in_orgdf = api.organizations_to_dataframe(api.get_organizations())

# Read the CSV files.  Change to API call if CSV's don't exist.
in_entries_df = pd.read_csv('data/entries_2024.csv')
in_events_df = pd.read_csv('data/events_2024.csv')


In [36]:
# Filter Orgs to State and Clubs in AU

# Create a regex pattern that matches either:
# - 3 letters, space, 1 letter (e.g., "ABC D")
# - 2-4 letters (e.g., "AB", "ABC", "ABCD")
# But excludes:
# - "NZ"
# - "AUS"
# - Anything starting with "CAS"
pattern = r'^(?!NZ$|AUS$|CAS)([A-Z]{2,3} [A-Z]|[A-Z]{2,4})$'

orgdf = in_orgdf[
    (in_orgdf['ShortName'].str.match(pattern, na=False)) & 
    (in_orgdf['CountryId'].astype(str).str.strip() == '36') # Australia only
].copy()

# Start with events dataframe and filter for disciplines 1 and 2
events_filtered = in_events_df[in_events_df['DisciplineIds'].isin(['1', '2'])].copy()

# Count entries per event
entries_per_event = in_entries_df.groupby('EventId').size().reset_index(name='EntryCount')

# Merge the entry counts back to events
events_with_entries = events_filtered.merge(
    entries_per_event,
    on='EventId',
    how='left'
).fillna(0)  # Fill NaN with 0 for events with no entries

# Now group by Organisation and Discipline to get:
# - Number of unique events
# - Average entry count per event
summary = events_with_entries.groupby(['OrganisationId', 'DisciplineIds']).agg(
    EventsCount=('EventId', 'nunique'),
    AvgEntryCountPerEvent=('EntryCount', 'mean')
).reset_index()

# Create pivot table with FootO and MTBO metrics side by side
pivot_summary = summary.pivot(
    index='OrganisationId',
    columns='DisciplineIds',
    values=['EventsCount', 'AvgEntryCountPerEvent']
).fillna(0)

# Flatten the column names
pivot_summary.columns = [f"{col[0]}_{col[1]}" for col in pivot_summary.columns]

# Sort by total events (sum of FootO and MTBO events)
pivot_summary['TotalEvents'] = pivot_summary['EventsCount_1'] + pivot_summary['EventsCount_2']
pivot_summary = pivot_summary.sort_values('TotalEvents', ascending=False)

# Drop the temporary total column
pivot_summary = pivot_summary.drop('TotalEvents', axis=1)

# Rename columns to be more descriptive
pivot_summary.columns = [
    'FootO_Events',
    'MTBO_Events',
    'FootO_AvgEntries',
    'MTBO_AvgEntries'
]

orgdf['OrganisationId'] = orgdf['OrganisationId'].astype(int)
pivot_summary.index = pivot_summary.index.astype(int)

# Merge the pivot table results with the organization dataframe
orgdf = orgdf.merge(
    pivot_summary,
    left_on='OrganisationId',
    right_index=True,
    how='left'
).fillna(0)  # Fill NaN values with 0 for organizations with no events

orgdf.to_csv('auorgs.csv', header=True)


In [37]:
in_events_df[in_events_df.OrganisationId==5]

,EventId,Name,StartDate,StartClock,FinishDate,FinishClock,EventClassificationId,EventStatusId,OrganisationId,WebURL,PunchingUnitType,DisciplineIds,EventRaceDistance,EventRaceId,EventRaceName,RaceDate,RaceClock,EventCenterX,EventCenterY,EventCenterUnit
69,20025,Orienteering NSW January Board Meeting,2024-01-23,08:30:00,2024-01-23,08:30:00,5,4,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20543,NaN,2024-01-23,08:30:00,NaN,NaN,NaN
175,20026,Orienteering NSW February Board Meeting,2024-02-27,08:30:00,2024-02-27,08:30:00,5,4,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20544,NaN,2024-02-27,08:30:00,NaN,NaN,NaN
334,20027,Orienteering NSW Annual General Meeting,2024-03-11,08:30:00,2024-03-11,08:30:00,5,5,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20545,NaN,2024-03-11,08:30:00,151.074482,-33.845997,WGS-84
335,20028,Orienteering NSW March Board Meeting,2024-03-19,08:30:00,2024-03-19,08:30:00,5,4,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20546,NaN,2024-03-19,08:30:00,NaN,NaN,NaN
394,20375,2024 NSW Schools Team Nominations,2024-03-17,13:00:00,2024-03-17,13:00:00,4,5,5,https://aoc2024.com.au/,SI,1,NaN,20909,NaN,2024-03-17,13:00:00,151.663851,-30.516645,WGS-84
437,19574,2024 NSW State League #3,2024-04-12,14:00:00,2024-04-12,14:00:00,3,10,5,https://onsw.asn.au/events/nsw-state-league,SI,1,NaN,20089,NaN,2024-04-12,14:00:00,NaN,NaN,NaN
448,19850,Bluebottles April Camp at Belanglo,2024-04-14,04:00:00,2024-04-17,03:00:00,5,5,5,NaN,SI,1,Long,20367,NaN,2024-04-14,04:00:00,150.259078,-34.532134,WGS-84
457,20029,Orienteering NSW April Board Meeting,2024-04-23,09:30:00,2024-04-23,09:30:00,5,4,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20547,NaN,2024-04-23,09:30:00,NaN,NaN,NaN
540,20707,Bluebottles Night Relay,2024-04-15,14:00:00,2024-04-15,14:00:00,4,9,5,NaN,SI,1,Sprint,21244,NaN,2024-04-15,14:00:00,NaN,NaN,NaN
570,20034,Orienteering NSW May General Meeting,2024-05-13,09:30:00,2024-05-13,09:30:00,5,5,5,https://onsw.asn.au/the-association/onsw-board...,manual,1,Long,20552,NaN,2024-05-13,09:30:00,151.074483,-33.846003,WGS-84
